In [86]:
import torch
import torch.nn as nn

# =================================================================

N = 256 # size of the array of Function F

# =================================================================

# Activation function for fitting
class F_function(nn.Module):
    """
    Activation function for fitting, works by saving points in a parametrized array
    """
    def __init__(self):
        super(F_function, self).__init__()
        self.force = nn.Parameter(torch.rand(N+1), requires_grad=True)
        
    def forward(self, X):
        """
        does an interpolation, in theory takes the input
        int i = v;
        and tries to do
        return F[v] 
        to find the corresponding value to that ´v´
        """

        if len(X) == 1:
            x, v = X[0], X[1]
        else:
            x, v = X[:,0], X[:,1]

        floor_v = torch.floor(v)
        ceil_v = (floor_v + 1).clamp(max=N) #adresses the overflow problem
        alpha = v - floor_v


        # return torch.tensor([x, (1 - alpha) * self.force[floor_v.int()] + alpha * self.force[ceil_v.int()]])
        return torch.stack([x, (1 - alpha) * self.force[floor_v.int()] + alpha * self.force[ceil_v.int()]]).T



## constructing a network with this component

In [75]:
import torch
import torch.nn as nn
# =================================================================

N = 256 # size of the array of Function F

# =================================================================    

import torch.nn.functional as F

class W_matrix(nn.Module):
    """ we need a linear thats not trainable """
    def __init__(self, dt):
        super(W_matrix, self).__init__()
        self.weights = torch.Tensor([[1, dt], [0, 1]])
        self.bias = torch.Tensor([0, 0])
        
    def forward(self, x): 
        return F.linear(x, self.weights, self.bias)

In [76]:
class diffNet(nn.Module):
    def __init__(self, depth):
        super(diffNet, self).__init__()
        layers = []
        layers.append(W_matrix(dt=1e-02))
        for i in range(depth):
            layers.append( F_function() )
            layers.append( W_matrix(dt=1e-02) )
        self.layers = nn.Sequential(*layers)
        
    def forward(self, X):
        return self.layers(X)


Generating the data from the simulations

In [77]:
import pandas as pd
df = pd.read_csv('.\data\songforce_v100\songforce.csv')
df.head()

data_worked = pd.DataFrame()
inicial = []
final = []

for index in range(10,len(df)-1):
    final.append(df.velocity[index])
    inicial.append(df.velocity[index - 10])

# print(inicial)
data_worked['inicial'] = inicial
data_worked['final'] = final

# seoarate 
Ndata = len(data_worked)
datatraining = data_worked[:202]
datavalidation = data_worked[202:250]
datatesting = data_worked[250:-1]

data_worked.head()


,inicial,final
0,100.000000,101.863643
1,100.185320,102.051319
2,100.370867,102.239240
3,100.556643,102.427406
4,100.742649,102.615821


In [78]:
X = torch.tensor( [ [0 for i in range(len(datatraining))], datatraining.inicial  ] ,  dtype=torch.float32).T
Y = torch.tensor( [ [0 for i in range(len(datatraining))], datatraining.final  ] ,  dtype=torch.float32)

X.size()

torch.Size([202, 2])

In [91]:
model = diffNet(depth=10)

model.forward(X)


tensor([[1.0451, 0.5921],
        [1.0477, 0.5921],
        [1.0504, 0.5921],
        [1.0530, 0.5921],
        [1.0557, 0.5921],
        [1.0584, 0.5921],
        [1.0603, 0.5921],
        [1.0619, 0.5921],
        [1.0634, 0.5921],
        [1.0650, 0.5921],
        [1.0665, 0.5921],
        [1.0684, 0.5921],
        [1.0711, 0.5921],
        [1.0738, 0.5921],
        [1.0765, 0.5921],
        [1.0792, 0.5921],
        [1.0819, 0.5921],
        [1.0835, 0.5921],
        [1.0850, 0.5921],
        [1.0865, 0.5921],
        [1.0880, 0.5921],
        [1.0895, 0.5921],
        [1.0916, 0.5921],
        [1.0939, 0.5921],
        [1.0963, 0.5921],
        [1.0986, 0.5921],
        [1.1010, 0.5921],
        [1.1030, 0.5921],
        [1.1045, 0.5921],
        [1.1061, 0.5921],
        [1.1077, 0.5921],
        [1.1092, 0.5921],
        [1.1108, 0.5921],
        [1.1122, 0.5921],
        [1.1136, 0.5921],
        [1.1151, 0.5921],
        [1.1165, 0.5921],
        [1.1180, 0.5921],
        [1.1

training the function with the  values

In [101]:
model.parameters() # accesing model parameters
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# I need to acces the model parameters for training
for paramA in zip(model.parameters()):
    paramA

constructing the loss compose function

In [102]:
def smooth_loss(force_params):
    summatory = 0.0
    for i in range(len(force_params)-1):
        summatory += ( force_params[i+1] - force_params[i] )**2
    return summatory

def physics_constrain(force_params):
    return force_params[0]



In [113]:
for paramA in model.parameters():
    print(len(paramA)) # access every force parameter at each layer
    # but we are getting extra parameters, because it is regenerating

257
257
257
257
257
257
257
257
257
257


remaking the diffNet

In [116]:
class diffNet(nn.Module):
    def __init__(self, depth):
        super(diffNet, self).__init__()

        w_mat = W_matrix(dt=1e-02)
        f_function = F_function()

        layers = []
        layers.append(w_mat)
        
        for i in range(depth):
            layers.append( f_function )
            layers.append( w_mat )
        self.layers = nn.Sequential(*layers)
        
    def forward(self, X):
        return self.layers(X)


In [119]:
model = diffNet(depth=10)

for paramA in model.parameters():
    print(len(paramA)) # access every force parameter at each layer
    # but we are getting extra parameters, because it is regenerating

257
